In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanics-data/train.csv
/kaggle/input/titanics-data/test.csv
/kaggle/input/titanics-data/gender_submission.csv


In [2]:
# 读取数据
train_df = pd.read_csv('/kaggle/input/titanics-data/train.csv')
test_df = pd.read_csv('/kaggle/input/titanics-data/test.csv')

In [3]:
# 观察数据
print(train_df.head())
print(train_df.info())  # 查看数据类型和缺失值情况

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
<c

In [4]:
#处理缺失值

# 填充 Age 缺失值（用中位数填充）
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)

# 填充 Fare 缺失值（用中位数填充）
train_df['Fare'].fillna(train_df['Fare'].median(), inplace=True)
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

# 填充 Embarked 缺失值（用众数填充）
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)

# 删除 Cabin 列（缺失值太多）
train_df.drop(columns=['Cabin'], inplace=True)
test_df.drop(columns=['Cabin'], inplace=True)

# 确保没有缺失值
print(train_df.isnull().sum())
print(test_df.isnull().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


<ipython-input-4-4659dfa4a9bb>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
<ipython-input-4-4659dfa4a9bb>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [5]:
#处理类别变量
# 转换 Sex（male -> 0, female -> 1）
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

# One-Hot 编码 Embarked
train_df = pd.get_dummies(train_df, columns=['Embarked'], drop_first=True)
test_df = pd.get_dummies(test_df, columns=['Embarked'], drop_first=True)

#提取 Title（从 Name 中提取称谓）
if 'Name' in train_df.columns:
    train_df['Title'] = train_df['Name'].str.extract('([A-Za-z]+)\.', expand=False)
    test_df['Title'] = test_df['Name'].str.extract('([A-Za-z]+)\.', expand=False)

    title_mapping = {
        'Mr': 1, 'Miss': 2, 'Mrs': 3, 'Master': 4, 
        'Dr': 5, 'Rev': 6, 'Col': 7, 'Major': 7, 'Mlle': 2, 'Mme': 3, 
        'Don': 8, 'Lady': 9, 'Countess': 9, 'Jonkheer': 8, 'Sir': 8, 'Capt': 7
    }
    train_df['Title'] = train_df['Title'].map(title_mapping).fillna(0)
    test_df['Title'] = test_df['Title'].map(title_mapping).fillna(0)

    # 删除 Name 列
    train_df.drop(columns=['Name'], inplace=True)
    test_df.drop(columns=['Name'], inplace=True)

#创建家庭大小特征
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

#票价取对数（处理异常值）
train_df['Fare_log'] = np.log1p(train_df['Fare'])
test_df['Fare_log'] = np.log1p(test_df['Fare'])

#删除无关特征
train_df.drop(columns=['Ticket', 'PassengerId', 'Fare'], inplace=True)
test_df.drop(columns=['Ticket', 'PassengerId', 'Fare'], inplace=True)

#确保所有数据都是数值类型
print(train_df.dtypes)
print(test_df.dtypes)

Survived        int64
Pclass          int64
Sex             int64
Age           float64
SibSp           int64
Parch           int64
Embarked_Q       bool
Embarked_S       bool
Title         float64
FamilySize      int64
Fare_log      float64
dtype: object
Pclass          int64
Sex             int64
Age           float64
SibSp           int64
Parch           int64
Embarked_Q       bool
Embarked_S       bool
Title         float64
FamilySize      int64
Fare_log      float64
dtype: object


In [6]:
# 目标变量
y = train_df['Survived']
X = train_df.drop(columns=['Survived'])  # 特征集

# 划分训练集和验证集
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# 训练随机森林模型
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 预测
y_pred = model.predict(X_valid)

# 计算准确率
accuracy = accuracy_score(y_valid, y_pred)
print(f'模型准确率: {accuracy:.4f}')


模型准确率: 0.8268


In [8]:
# 在测试集上进行预测
test_predictions = model.predict(test_df)

# 读取原始测试集，获取 PassengerId
test_original = pd.read_csv('/kaggle/input/titanics-data/test.csv')

# 生成提交文件
submission = pd.DataFrame({'PassengerId': test_original['PassengerId'], 'Survived': test_predictions})
submission.to_csv('submission.csv', index=False)

print("预测结果已保存为 submission.csv")


预测结果已保存为 submission.csv
